In [ ]:
# default_exp seq_core
# default_cls_lvl 3

# Codestructure

1. Extract File Paths
2. Convert Paths to Sequence Objects (TensorSequence, TensorScalars, TensorSequence) <-> (input,input,output)
2. Data Manipulation
    1. Sequence pruning
    2. Sequence Resampling
4. Split Sequence Objects in Train, Validation
3. (Create Windows out of Sequence Objects)
5. Noise Injection in Training Dataset Input
5. Normalize Input 

In [ ]:
#export
from fastai2.data.all import *
import h5py

## 1. Extract File Paths
Der erste Schritt kann mit get_files von fastai2 erledigt werden. Man kann noch optional eine get_hdf_files() Funktion schreiben.

In [ ]:
# hdf_files = get_files('/mnt/Data/Batterydata/Measurements/',extensions='.hdf5',recurse=True)
hdf_files = get_files('/mnt/Data/Batterydata/Measurements/HE4_1/',extensions='.hdf5',recurse=False)
len(hdf_files),hdf_files[0]

(6,
 PosixPath('/mnt/Data/Batterydata/Measurements/HE4_1/HE4_1_PulseTest0.hdf5'))

## 2. Convert Paths to Sequence Objects
Der Pfad wird unter Angabe der Spaltennamen in Sequenzen und Skalare Werte umgewandelt, um so am Ende ein 3-Tupel zu erhalten aus:
- (Sequence, Scalar, Sequence) <-> (input,input,output)

In [ ]:
#export
def hdf2sequence(hdf_path,c_names):
    with h5py.File(hdf_path,'r') as f:
#         import pdb; pdb.set_trace()
        l_array = [f[n][:][:,None] for n in c_names]
        seq = np.concatenate(l_array,axis=1)
        return seq

In [ ]:
hdf2sequence(hdf_files[0],['current','voltage']).shape

(64866, 2)

Die Funktion lässt sich mittels Pipeline auf eine Liste von Quellobjekten (hier Pfade) anwenden 

In [ ]:
pipe = Pipeline(partial(hdf2sequence,c_names=['current','voltage']))

In [ ]:
res_pipe = pipe(hdf_files)
len(res_pipe), res_pipe[0][0]

(6, array([4.001108, 3.735982], dtype=float32))

In [ ]:
#export
def hdf2scalars(hdf_path,c_names):
    with h5py.File(hdf_path,'r') as f:
#         import pdb; pdb.set_trace()
#         l_array = [f[n][:][:,None] for n in c_names]
#         seq = np.concatenate(l_array,axis=1)
        return None

### Tensor Tuple erstellen

In [ ]:
#export
class Hdf2SeqSeq(Transform):
    def __init__(self, seq_inp,scal_inp, seq_out): self.seq_inp,self.scal_inp,self.seq_out = seq_inp,scal_inp,seq_out
    def encodes(self, o): return (hdf2sequence(o,self.seq_inp),
                                  hdf2scalars(o,self.scal_inp),
                                  hdf2sequence(o,self.seq_out))
    def decodes(self, x): return SequenceItem(x)

class Hdf2SeqScal(Transform):
    def __init__(self, seq_inp,scal_inp, scal_out): self.seq_inp,self.scal_inp,self.scal_out = seq_inp,scal_inp,scal_out
    def encodes(self, o): return (hdf2sequence(o,self.seq_inp),
                                  hdf2scalars(o,self.scal_inp),
                                  hdf2scalars(o,self.scal_out))
    def decodes(self, x): return SequenceItem(x) 

In [ ]:
hdf2seq = Pipeline(Hdf2SeqSeq(['current','voltage'],None,['voltage']))

items = hdf2seq(hdf_files)
len(items),items[0]

(6, (array([[  4.001108 ,   3.735982 ],
         [  4.001108 ,   3.7363622],
         [  4.001108 ,   3.7367425],
         ...,
         [-14.899409 ,   2.505974 ],
         [-14.899409 ,   2.502833 ],
         [-14.899141 ,   2.4996917]], dtype=float32), None, array([[3.735982 ],
         [3.7363622],
         [3.7367425],
         ...,
         [2.505974 ],
         [2.502833 ],
         [2.4996917]], dtype=float32)))

### SequenceItem
Damit die Sequenz visualisiert werden kann und auch dritte Informationen gespeichert werden können, wird eine Klasse erstellt 

In [ ]:
#export

#TODO: Pruefen ob Zielgroesse 'self[2]' sequenz ist
class SequenceItem(Tuple):
    def show(self, ctx=None, **kwargs): 
        plt.figure()
        plt.plot(self[2])

SequenceItem ist nur für die Darstellung eines Tupels von Sequenzen zuständig. Es muss zwischen Skalaren und Vektoriellen Zielgrößen unterschieden werden.

In [ ]:
#export
class SeqTfm(Transform):
    def decodes(self, x): return SequenceItem(x)

In [ ]:
# compose_tfms??
# Pipeline??

SequenceTfm erstellt ein SequenceItem beim decoding für die spätere Darstellung.

## 5. Create Windows
Aus einer langen Sequenz werden mehrere kurze Sequenzen extrahiert um so verschiedene Teile zu gleicher Zeit dem Model zu zeigen.

Dies geschieht auf Tuple level und ist deshalb kein TupleTransform

In [ ]:
obj = items[0]
len(obj[2]),obj[2]

(64866, array([[3.735982 ],
        [3.7363622],
        [3.7367425],
        ...,
        [2.505974 ],
        [2.502833 ],
        [2.4996917]], dtype=float32))

Testen ob bei vervielfältigung sich der Speicherbedarf massiv ändert

In [ ]:
temp = [obj]*100000

Untersuchung mittels Speicherbedarf bei htop zeigt, dass sich der Speicherbedarf selbst bei 10^5 facher Liste nicht um mehr als 10 MB ändert. Das ist die Darstellungsgrenze

In [ ]:
#export

def createWindows(x):
    win_size = 100
    x_seq = x[0]
    n_win = x_seq.shape[0]//win_size
    win_list = [x]*n_win
    for i,win in enumerate(win_list):
        win_list[i]=(
            win[0][i*win_size:(i+1)*win_size],
            win[1],
            win[2][i*win_size:(i+1)*win_size])
    return L(win_list)

In [ ]:
tst = createWindows(obj)

In [ ]:
Pipeline??

In [ ]:
tst[1][0].shape

(100, 2)

In [ ]:
pipe_tst = Pipeline(createWindows)
lst = pipe_tst(items)

95.5 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
lst

((#648) [(array([[4.001108 , 3.735982 ],
        [4.001108 , 3.7363622],
        [4.001108 , 3.7367425],
        [4.001108 , 3.7371225],
        [4.001108 , 3.7375028],
        [4.001108 , 3.737883 ],
        [4.001108 , 3.7382634],
        [4.001108 , 3.7386436],
        [4.001108 , 3.7390237],
        [4.001108 , 3.739404 ],
        [4.001108 , 3.7397842],
        [4.001162 , 3.7400982],
        [4.0012155, 3.7404125],
        [4.001269 , 3.7407267],
        [4.0013227, 3.7410407],
        [4.001376 , 3.741355 ],
        [4.00143  , 3.741669 ],
        [4.0014834, 3.7419832],
        [4.0015373, 3.7422972],
        [4.0015907, 3.7426114],
        [4.0016446, 3.7429254],
        [4.001698 , 3.743157 ],
        [4.001752 , 3.7433884],
        [4.0018053, 3.7436197],
        [4.001859 , 3.7438512],
        [4.0019126, 3.7440827],
        [4.0019665, 3.7443142],
        [4.00202  , 3.7445457],
        [4.002074 , 3.744777 ],
        [4.002127 , 3.7450085],
        [4.002181 , 3.74524  ],

## 6. Split Items
Splitting ordnet die Quelldateien Training und Validierung zu. Dafür können die Fastai Implementierungen teilweise verwendet werden.

Splitting innerhalb einer Sequenzen sollte in der Praxis nur dann geschehen wenn eine einzige Sequenz vorhanden ist. Diese kann dann vorher manuell geteilt werden.



# Export

In [ ]:
# #hide
# from nbdev.export import *
# notebook2script()